In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# training metrics

In [8]:
from google.cloud import aiplatform
model = aiplatform.Model('projects/351298396653/locations/us-central1/models/7488977106433474560@1')

In [9]:
model.list_model_evaluations()

[]

In [5]:
from vertexai.preview.tuning import sft
job = sft.SupervisedTuningJob("projects/351298396653/locations/us-central1/tuningJobs/9131142287097593856")

In [6]:
job.to_dict()

{'name': 'projects/351298396653/locations/us-central1/tuningJobs/9131142287097593856',
 'tunedModelDisplayName': 'gemini-1.0-pro-002:sweep',
 'baseModel': 'gemini-1.0-pro-002',
 'supervisedTuningSpec': {'trainingDatasetUri': 'gs://cloud-ai-platform-6e5ab5cb-3fca-49e0-a42c-ce00ed910490/instrospection-astra/gemini-1.0-pro-002-2024-05-14-22-35-53_train_dataset.jsonl',
  'validationDatasetUri': 'gs://cloud-ai-platform-6e5ab5cb-3fca-49e0-a42c-ce00ed910490/instrospection-astra/gemini-1.0-pro-002-2024-05-14-22-35-55_val_dataset.jsonl',
  'hyperParameters': {}},
 'state': 'JOB_STATE_SUCCEEDED',
 'createTime': '2024-05-15T02:35:57.294147Z',
 'startTime': '2024-05-15T02:35:57.332828Z',
 'endTime': '2024-05-15T03:31:02.378289Z',
 'updateTime': '2024-05-15T03:31:02.378289Z',
 'experiment': 'projects/351298396653/locations/us-central1/metadataStores/default/contexts/f05e1483-8179-4f7b-9e80-bced91a6bf09',
 'tunedModel': {'model': 'projects/351298396653/locations/us-central1/models/748897710643347456

In [2]:
from google.cloud import aiplatform

# Replace with your project and region
project = "roots-api-1475521819980"
region = "us-central1"

# Replace with the model and tuning run IDs
model_id = "5495612003298836480"
tuning_run_id = "9131142287097593856"
# model_id = "projects/351298396653/locations/us-central1/endpoints/5495612003298836480"
# tuning_run_id = "projects/351298396653/locations/us-central1/tuningJobs/9131142287097593856"

# Create the Vertex AI client
client = aiplatform.gapic.ModelServiceClient(
    client_options={"api_endpoint": f"{region}-aiplatform.googleapis.com"}
)

# Get the tuning run
tuning_run = client.get_model_evaluation(name=f"projects/{project}/locations/{region}/models/{model_id}/evaluations/{tuning_run_id}")

# Access the training metrics
training_metrics = tuning_run.metrics

# Print the training metrics
print(training_metrics)

NotFound: 404 The Model does not exist.

# upload data, finetune, deploy

In [39]:
from evals.apis.finetuning.run import upload_file, wait_until_finetune_job_is_ready
from evals.utils import load_jsonl


In [53]:
from pathlib import Path
data_path = Path('/Users/milesturpin/Dev/nyu/introspection_self_prediction_astra/exp/finetuning/finetuning_demo3/gpt-3.5-turbo/train_dataset.jsonl')
samples = load_jsonl(data_path)


In [82]:

from evals.apis.finetuning.run import FineTuneHyperParams, FineTuneParams


ft = FineTuneHyperParams(learning_rate_multiplier=1.0, n_epochs=1, batch_size=None)
params = FineTuneParams(model='gemini-1.0-pro-002', suffix='test-ft', hyperparameters=ft)


file_id = upload_file(data_path, params)


Starting file upload.
gemini-1.0-pro-002-2024-05-09-13-38-26_train_dataset.jsonl
Uploading to gcloud
File temp_filtered.jsonl uploaded to instrospection-astra/gemini-1.0-pro-002-2024-05-09-13-38-26_train_dataset.jsonl.
File ID is gsutil URI: gs://cloud-ai-platform-6e5ab5cb-3fca-49e0-a42c-ce00ed910490/instrospection-astra/gemini-1.0-pro-002-2024-05-09-13-38-26_train_dataset.jsonl
Uploaded file.
gemini-1.0-pro-002-2024-05-09-13-38-26_train_dataset.jsonl
gs://cloud-ai-platform-6e5ab5cb-3fca-49e0-a42c-ce00ed910490/instrospection-astra/gemini-1.0-pro-002-2024-05-09-13-38-26_train_dataset.jsonl


In [83]:
from evals.apis.finetuning.run import FineTuneHyperParams, FineTuneParams
ft = FineTuneHyperParams(learning_rate_multiplier=1.0, n_epochs=1, batch_size=None)
params = FineTuneParams(model='gemini-1.0-pro-002', suffix='test-ft', hyperparameters=ft)

In [84]:
from evals.apis.finetuning.run import queue_finetune

# file_id = 'instrospection-astra/gemini-1.0-pro-002-2024-05-07-19-22-53_train_dataset.jsonl'
finetune_job_resp = queue_finetune(
    file_id=file_id,
    model=params.model,
    suffix=params.suffix, # TODO: is suffix usually set?
    retries=1, 
    retry_time=1, 
    hyperparameters=params.hyperparameters)      

Creating SupervisedTuningJob


SupervisedTuningJob created. Resource name: projects/351298396653/locations/us-central1/tuningJobs/1444862217106227200
To use this SupervisedTuningJob in another session:
tuning_job = sft.SupervisedTuningJob('projects/351298396653/locations/us-central1/tuningJobs/1444862217106227200')
View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/1444862217106227200?project=351298396653
Started finetune job. {'name': 'projects/351298396653/locations/us-central1/tuningJobs/1444862217106227200', 'tunedModelDisplayName': 'gemini-1.0-pro-002:test-ft', 'baseModel': 'gemini-1.0-pro-002', 'supervisedTuningSpec': {'trainingDatasetUri': 'gs://cloud-ai-platform-6e5ab5cb-3fca-49e0-a42c-ce00ed910490/instrospection-astra/gemini-1.0-pro-002-2024-05-09-13-38-26_train_dataset.jsonl', 'hyperParameters': {'epochCount': '1', 'learningRateMultiplier': 1.0}}, 'state': 'JOB_STATE_PENDING', 'createTime': '2024-05-09T17:38:49.476069Z', 'updateTime': '202

In [88]:
result = wait_until_finetune_job_is_ready(finetune_job_resp)

/Users/milesturpin/Dev/nyu/introspection_self_prediction_astra/evals/run_finetuning.py:31: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="conf", config_name="config_finetuning_run")
[autoreload of evals.utils failed: Traceback (most recent call last):
  File "/Users/milesturpin/Dev/nyu/introspection_self_prediction_astra/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/milesturpin/Dev/nyu/introspection_self_prediction_astra/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/usr/local/Cellar/python@3.11/3.11.9/Frameworks/Python.framework/Versions/3.11/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<

Running Gemini SFT job:
 {'name': 'projects/351298396653/locations/us-central1/tuningJobs/1444862217106227200', 'tunedModelDisplayName': 'gemini-1.0-pro-002:test-ft', 'baseModel': 'gemini-1.0-pro-002', 'supervisedTuningSpec': {'trainingDatasetUri': 'gs://cloud-ai-platform-6e5ab5cb-3fca-49e0-a42c-ce00ed910490/instrospection-astra/gemini-1.0-pro-002-2024-05-09-13-38-26_train_dataset.jsonl', 'hyperParameters': {'epochCount': '1', 'learningRateMultiplier': 1.0}}, 'state': 'JOB_STATE_SUCCEEDED', 'createTime': '2024-05-09T17:38:49.476069Z', 'startTime': '2024-05-09T17:38:49.510007Z', 'endTime': '2024-05-09T18:07:40.114368Z', 'updateTime': '2024-05-09T18:07:40.114368Z', 'experiment': 'projects/351298396653/locations/us-central1/metadataStores/default/contexts/654be68d-316b-4b18-997f-cb8f2a5887de', 'tunedModel': {'model': 'projects/351298396653/locations/us-central1/models/1915104064492797952@1', 'endpoint': 'projects/351298396653/locations/us-central1/endpoints/4261836811631853568'}, 'tuningD

In [72]:
model_id = result.fine_tuned_model

In [79]:
model_id

'projects/351298396653/locations/us-central1/endpoints/1362644551512096768'

In [74]:
from evals.apis.finetuning.run import deploy_gemini_model


deploy_gemini_model(model_id, finetune_job_resp, 'test-ft')

Deploying model to Endpoint : projects/351298396653/locations/us-central1/endpoints/1362644551512096768
Model does not support deployment. See https://cloud.google.com/vertex-ai/docs/reference/rpc/google.cloud.aiplatform.v1#google.cloud.aiplatform.v1.Model.FIELDS.repeated.google.cloud.aiplatform.v1.Model.DeploymentResourcesType.google.cloud.aiplatform.v1.Model.supported_deployment_resources_types
Deploy Endpoint model backing LRO: projects/351298396653/locations/us-central1/endpoints/1362644551512096768/operations/8048985350854410240
Endpoint model deployed. Resource name: projects/351298396653/locations/us-central1/endpoints/1362644551512096768
gemini-1.0-pro-002:test-ft
projects/351298396653/locations/us-central1/models/2155483694603698176
gemini-1.0-pro-002:test-ft
projects/351298396653/locations/us-central1/endpoints/1362644551512096768


In [77]:

import vertexai
from vertexai.generative_models import GenerativeModel
# project_id='roots-api-1475521819980'
# location='us-central1'
# vertexai.init(project=project_id, location=location)

generation_config = {
    "max_output_tokens": 100,
    "temperature": 0.9,
    "top_p": 1,
}
model = GenerativeModel('projects/351298396653/locations/us-central1/endpoints/1362644551512096768')
response = model.generate_content(['hello'], generation_config=generation_config)

In [87]:
from argparse import Namespace

from omegaconf import OmegaConf
from evals.run_finetuning import create_finetuned_model_config

cfg = Namespace(study_name='finetuning_demo2')


OmegaConf.clear_resolver('sanitize')
create_finetuned_model_config(cfg, 'projects/351298396653/locations/us-central1/endpoints/1362644551512096768', )

projects_351298396653_locations_us-central1_endpoints_1362644551512096768


'finetuned/finetuning_demo2/projects_351298396653_locations_us-central1_endpoints_1362644551512096768'

In [78]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Hello! 👋 How can I assist you today? \n\nIs there anything specific you\'d like to talk about or would you prefer to start with a general question?"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0339719951
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0241914857
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.0347496271
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0221588686
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.0512724183
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0137953646
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.140808165
    severity: HARM_SE

In [51]:
from vertexai.preview.tuning import sft
job = sft.SupervisedTuningJob('projects/351298396653/locations/us-central1/tuningJobs/1125695981795409920')

In [75]:
job.to_dict()

{'name': 'projects/351298396653/locations/us-central1/tuningJobs/1125695981795409920',
 'tunedModelDisplayName': 'gemini-1.0-pro-002:test-ft',
 'baseModel': 'gemini-1.0-pro-002',
 'supervisedTuningSpec': {'trainingDatasetUri': 'gs://cloud-ai-platform-6e5ab5cb-3fca-49e0-a42c-ce00ed910490/instrospection-astra/gemini-1.0-pro-002-2024-05-07-20-38-11_train_dataset.jsonl',
  'hyperParameters': {'epochCount': '1', 'learningRateMultiplier': 1.0}},
 'state': 'JOB_STATE_FAILED',
 'createTime': '2024-05-08T00:38:15.872906Z',
 'startTime': '2024-05-08T00:38:15.922074Z',
 'endTime': '2024-05-08T00:55:56.157725Z',
 'updateTime': '2024-05-08T00:55:56.157725Z',
 'error': {'code': 13,
  'message': 'Internal error occurred. Contact Vertex AI.'},
 'experiment': 'projects/351298396653/locations/us-central1/metadataStores/default/contexts/1e16d05d-364e-4dc7-8abe-90a252448d80',
 'tuningDataStats': {'supervisedTuningDataStats': {'tuningDatasetExampleCount': '16',
   'totalTuningCharacterCount': '6864',
   't

In [11]:
data_path.name

'train_dataset.jsonl'

### deploy

In [59]:
from evals.utils import GCLOUD_LOCATION, GCLOUD_PROJECT
from google.cloud import aiplatform

aiplatform.init(project=GCLOUD_PROJECT, location=GCLOUD_LOCATION)

display_name = f"gemini-1.0-pro-002:finetune-test"
endpoint = aiplatform.Endpoint.create(
        display_name=display_name, # TODO: use same name as elsewhere?
        project=GCLOUD_PROJECT,
        location=GCLOUD_LOCATION,
    )

Creating Endpoint
Create Endpoint backing LRO: projects/351298396653/locations/us-central1/endpoints/1216277563622555648/operations/5345699674525270016
Endpoint created. Resource name: projects/351298396653/locations/us-central1/endpoints/1216277563622555648
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/351298396653/locations/us-central1/endpoints/1216277563622555648')


In [63]:
sftjob = sft.SupervisedTuningJob('projects/351298396653/locations/us-central1/tuningJobs/4314075193183043584')

In [69]:
sftjob.tuned_model_endpoint_name
sftjob.tuned_model_name

'projects/351298396653/locations/us-central1/endpoints/2082604765379821568'

In [65]:
sftjob.to_dict()

{'name': 'projects/351298396653/locations/us-central1/tuningJobs/4314075193183043584',
 'tunedModelDisplayName': 'SupervisedTuningJob 2024-05-02 21:35:45.936798',
 'baseModel': 'gemini-1.0-pro-002',
 'supervisedTuningSpec': {'trainingDatasetUri': 'gs://cloud-samples-data/ai-platform/generative_ai/sft_train_data.jsonl',
  'hyperParameters': {}},
 'state': 'JOB_STATE_SUCCEEDED',
 'createTime': '2024-05-03T01:35:46.563985Z',
 'startTime': '2024-05-03T01:35:46.622218Z',
 'endTime': '2024-05-03T02:16:23.756855Z',
 'updateTime': '2024-05-03T02:16:23.756855Z',
 'experiment': 'projects/351298396653/locations/us-central1/metadataStores/default/contexts/9a8aed9c-4339-453c-8bd4-e798e7a7b90e',
 'tunedModel': {'model': 'projects/351298396653/locations/us-central1/models/717850251054022656@1',
  'endpoint': 'projects/351298396653/locations/us-central1/endpoints/2082604765379821568'},
 'tuningDataStats': {'supervisedTuningDataStats': {'tuningDatasetExampleCount': '500',
   'totalTuningCharacterCount'

In [ ]:
model = aiplatform.Model(model_name=model_name)

In [61]:
endpoint.resource_name

'projects/351298396653/locations/us-central1/endpoints/1216277563622555648'

# gcloud

In [1]:
import vertexai
from vertexai.language_models import TextGenerationModel

https://cloud.google.com/vertex-ai/generative-ai/docs/samples/generativeaionvertexai-gemini-pro-config-example

In [2]:
# import vertexai.generativeai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models


project_id='roots-api-1475521819980'
location='us-central1'

vertexai.init(project=project_id, location=location)

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

generation_config = {
    "max_output_tokens": 100,
    "temperature": 0.9,
    "top_p": 1,
}
model = GenerativeModel("gemini-1.0-pro-001")
responses = model.generate_content(
      ['hello'],
      generation_config=generation_config,
      safety_settings=safety_settings,
  )

print(responses)




candidates {
  content {
    role: "model"
    parts {
      text: "Hello and welcome to my profile! I\'m a multi-modal AI model, developed by Google. I\'m designed to help with a wide range of writing tasks, from creative writing to technical documentation. My capabilities include:\n\n* **Language Generation:** I can generate text in a variety of styles and tones, from casual to formal, and from creative to informative. This makes me a great tool for writing marketing materials, articles, stories, and more.\n* **Language Translation:** I"
    }
  }
  finish_reason: MAX_TOKENS
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0214097705
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0345536247
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.00703904592
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.040162582
  }
  safety_rat

In [3]:
# projects/351298396653/locations/us-central1/endpoints/985415306161684480",
# )

model = GenerativeModel('projects/351298396653/locations/us-central1/endpoints/3934710112135938048')

response = model.generate_content(['hello'], generation_config=generation_config, safety_settings=safety_settings)

In [4]:
sft_tuning_job.has_ended

NameError: name 'sft_tuning_job' is not defined

### chat session

In [17]:
from vertexai.generative_models import GenerativeModel, ChatSession

model = GenerativeModel(model_name="gemini-1.0-pro-002", system_instruction=[
        "Speak like shakespeare.",
    ],)
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

prompt = "Hello."
print(get_chat_response(chat, prompt))

prompt = "What are all the colors in a rainbow?"
print(get_chat_response(chat, prompt))

prompt = "Why does it appear when it rains?"
print(get_chat_response(chat, prompt))

Well met, friend. What brings thee here today?
Is there aught that thou wouldst ask of me, 
Or some tale thou wishest me to tell?
Speak thy mind, and I shall do my best to serve.
**In the bright spectrum of the rainbow's arch,**

**Seven hues dance, a vibrant, shimmering march.**

**First, red doth blaze, a passionate fire,**

**Orange glows, a sunset's warm desire.**

**Yellow's brilliance, like sun's golden ray,**

**Green's verdant cloak, where life doth hold sway.**

**Blue's tranquil depth, like ocean's boundless hold,**

**Indigo's mystic shade, a tale untold.**

**And lastly, violet's ethereal grace,**

**A final note, in celestial space.**
When sunlight meets the raindrops' tearful fall,
A wondrous magic paints the heavens tall.
The raindrops act as prisms in the air,
Refracting light, a spectrum bright and rare.

Each drop, a tiny lens, does bend the ray,
Splitting its colors, chasing shadows away.
Red, orange, yellow, green, and blue,
And indigo and violet, a wondrous crew.



### finetuning

In [66]:
from google.cloud import storage

storage_client = storage.Client(project='roots-api-1475521819980')
bucket = storage_client.bucket('cloud-ai-platform-6e5ab5cb-3fca-49e0-a42c-ce00ed910490')
blob = bucket.blob('test/test.txt')


In [67]:
blob.upload_from_filename('/Users/milesturpin/Dev/nyu/introspection_self_prediction_astra/exp/smol_sweep_demo3/divergent_strings_number_triplets.csv')

In [26]:
from vertexai.preview import tuning
from vertexai.preview.tuning import sft

sft_tuning_job = sft.train(
    source_model='gemini-1.0-pro-002',
    train_dataset="gs://cloud-samples-data/ai-platform/generative_ai/sft_train_data.jsonl",
    validation_dataset="gs://cloud-samples-data/ai-platform/generative_ai/sft_validation_data.jsonl",
    epochs=4,
    learning_rate_multiplier=1.0,
  tuned_model_display_name='tuned_gemini_pro'
)
sft_tuning_job.to_dict()

Creating SupervisedTuningJob


SupervisedTuningJob created. Resource name: projects/351298396653/locations/us-central1/tuningJobs/944678984468135936
To use this SupervisedTuningJob in another session:
tuning_job = sft.SupervisedTuningJob('projects/351298396653/locations/us-central1/tuningJobs/944678984468135936')
View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/944678984468135936?project=351298396653


{'name': 'projects/351298396653/locations/us-central1/tuningJobs/944678984468135936',
 'tunedModelDisplayName': 'tuned_gemini_pro',
 'baseModel': 'gemini-1.0-pro-002',
 'supervisedTuningSpec': {'trainingDatasetUri': 'gs://cloud-samples-data/ai-platform/generative_ai/sft_train_data.jsonl',
  'validationDatasetUri': 'gs://cloud-samples-data/ai-platform/generative_ai/sft_validation_data.jsonl',
  'hyperParameters': {'epochCount': '4', 'learningRateMultiplier': 1.0}},
 'state': 'JOB_STATE_PENDING',
 'createTime': '2024-05-02T22:03:56.308353Z',
 'updateTime': '2024-05-02T22:03:56.308353Z'}

In [61]:
sft_tuning_job.to_dict()

{'name': 'projects/351298396653/locations/us-central1/tuningJobs/4314075193183043584',
 'tunedModelDisplayName': 'SupervisedTuningJob 2024-05-02 21:35:45.936798',
 'baseModel': 'gemini-1.0-pro-002',
 'supervisedTuningSpec': {'trainingDatasetUri': 'gs://cloud-samples-data/ai-platform/generative_ai/sft_train_data.jsonl',
  'hyperParameters': {}},
 'state': 'JOB_STATE_PENDING',
 'createTime': '2024-05-03T01:35:46.563985Z',
 'updateTime': '2024-05-03T01:35:46.563985Z'}

In [27]:
sft_tuning_job = sft.SupervisedTuningJob('projects/351298396653/locations/us-central1/tuningJobs/944678984468135936')

In [28]:
sft_tuning_job.tuned_model_endpoint_name

'projects/351298396653/locations/us-central1/endpoints/985415306161684480'

In [29]:
model = GenerativeModel(sft_tuning_job.tuned_model_endpoint_name)

In [30]:
model.generate_content(["hello"])

candidates {
  content {
    role: "model"
    parts {
      text: "Hello! 👋 How can I help you today?"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0263052788
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0406167768
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.0424038284
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0271692332
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.0538991578
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0169149134
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.0982522294
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0280607399
  }
}
usage_metadata {
  prompt_token_count: 1
  candidates_t

In [ ]:
sft.SupervisedTuningJob('')

In [41]:
sft_tuning_job = sft.train(
    source_model="gemini-1.0-pro-002",
    train_dataset="gs://cloud-samples-data/ai-platform/generative_ai/sft_train_data.jsonl"
)
# Polling for job completion
# while not sft_tuning_job.has_ended:
#     time.sleep(60)
#     sft_tuning_job.refresh()

# print(sft_tuning_job.tuned_model_name)
# print(sft_tuning_job.tuned_model_endpoint_name)
# print(sft_tuning_job.experiment)

Creating SupervisedTuningJob
SupervisedTuningJob created. Resource name: projects/351298396653/locations/us-central1/tuningJobs/4314075193183043584
To use this SupervisedTuningJob in another session:
tuning_job = sft.SupervisedTuningJob('projects/351298396653/locations/us-central1/tuningJobs/4314075193183043584')
View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/4314075193183043584?project=351298396653


In [68]:
sft_tuning_job.to_dict()

{'name': 'projects/351298396653/locations/us-central1/tuningJobs/4314075193183043584',
 'tunedModelDisplayName': 'SupervisedTuningJob 2024-05-02 21:35:45.936798',
 'baseModel': 'gemini-1.0-pro-002',
 'supervisedTuningSpec': {'trainingDatasetUri': 'gs://cloud-samples-data/ai-platform/generative_ai/sft_train_data.jsonl',
  'hyperParameters': {}},
 'state': 'JOB_STATE_PENDING',
 'createTime': '2024-05-03T01:35:46.563985Z',
 'updateTime': '2024-05-03T01:35:46.563985Z'}

In [43]:
print(sft_tuning_job.tuned_model_endpoint_name)

In [44]:
sft_tuning_job.tuned_model_endpoint_name

''

# debug OAI inference

In [1]:
import pandas as pd

In [8]:
from evals.apis.inference.api import InferenceAPI
from evals.apis.inference.openai.chat import OpenAIChatModel
from evals.utils import setup_environment

In [5]:
setup_environment()
inference_api  = InferenceAPI()

In [40]:
openai = OpenAIChatModel(frac_rate_limit=0,
                        #   organization='DEFAULT_ORG'
                          organization='org-4L2GWAH28buzKOIhEAb3L5aq'
                         )

In [41]:
import asyncio
from evals.data_models.messages import ChatMessage, Prompt

message = ChatMessage(role='user', content='hi')
prompt = Prompt(messages=[message])


In [46]:
loop = asyncio.get_running_loop()

In [48]:
result = await openai._make_api_call(prompt, model_id='gpt-3.5-turbo', start_time=0)

In [52]:
result = await openai._get_dummy_response_header('gpt-3.5-turbo')

In [53]:
result

{'Date': 'Fri, 19 Apr 2024 19:43:18 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'Cache-Control': 'no-cache, must-revalidate', 'openai-model': 'gpt-3.5-turbo-0125', 'openai-organization': 'nyu-arg', 'openai-processing-ms': '457', 'openai-version': '2020-10-01', 'strict-transport-security': 'max-age=15724800; includeSubDomains', 'x-ratelimit-limit-requests': '10000', 'x-ratelimit-limit-tokens': '2000000', 'x-ratelimit-remaining-requests': '9999', 'x-ratelimit-remaining-tokens': '1999980', 'x-ratelimit-reset-requests': '6ms', 'x-ratelimit-reset-tokens': '0s', 'x-request-id': 'req_024505a118ab36791dc3ea527ac4382d', 'CF-Cache-Status': 'DYNAMIC', 'Set-Cookie': '__cf_bm=bnyfAbNnMqoMepP417U6leuyY98RgX8KvVftH3y_OMQ-1713555798-1.0.1.1-G7Fu3wSqkzHwOW73ls8llcNLw1fAAdXhn07AY.judEPU_QqGmKNxyTuS.cW2gd.eeSHC5GykyUCNrPZs1cWpyQ; path=/; expires=Fri, 19-Apr-24 20:13:18 GMT; domain=.api.openai.com; HttpOnly; Secu

In [37]:
import openai




In [38]:
openai.__version__

'0.28.0'

In [39]:
openai.ChatCompletion.create(model='gpt-3.5-turbo', messages= [
      {
        "role": "system",
        "content": "You are a helpful assistant."
      },
      {
        "role": "user",
        "content": "Hello!"
      }
    ])

<OpenAIObject chat.completion id=chatcmpl-9FoK4JDfpD7lCaSMNSLPBXn93Tkdu at 0x1423cd010> JSON: {
  "id": "chatcmpl-9FoK4JDfpD7lCaSMNSLPBXn93Tkdu",
  "object": "chat.completion",
  "created": 1713555388,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 19,
    "completion_tokens": 9,
    "total_tokens": 28
  },
  "system_fingerprint": "fp_d9767fc5b9"
}

In [24]:

result = await openai._make_api_call(prompt,'gpt-3.5-turbo', start_time=0)

AttributeError: module 'openai' has no attribute '_make_api_call'